<a href="https://colab.research.google.com/github/longqua69/practice-deep-learning/blob/44_tf_data_pipeline/44_tf_data_pipeline_exercise/44_tf_data_pipeline_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf

# Create dataset

## Clone the entire repository and navigate to the exercise directory

In [28]:
# Clone the entire repo
!git clone -l -s https://github.com/longqua69/practice-deep-learning.git working_dir
%cd working_dir
!ls -la

Cloning into 'working_dir'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 54 (delta 12), reused 21 (delta 1), pack-reused 0
Unpacking objects: 100% (54/54), done.
/content/working_dir/44_tf_data_pipeline_exercise/working_dir
total 28
drwxr-xr-x 5 root root 4096 Oct 24 07:51 .
drwxr-xr-x 4 root root 4096 Oct 24 07:51 ..
drwxr-xr-x 3 root root 4096 Oct 24 07:51 44_tf_data_pipeline_exercise
drwxr-xr-x 2 root root 4096 Oct 24 07:51 6_gradient_descent_exercise
drwxr-xr-x 8 root root 4096 Oct 24 07:51 .git
-rw-r--r-- 1 root root 1065 Oct 24 07:51 LICENSE
-rw-r--r-- 1 root root  107 Oct 24 07:51 README.md


In [29]:
# Navigate to the exercise working directory
%cd 44_tf_data_pipeline_exercise
!ls -la

/content/working_dir/44_tf_data_pipeline_exercise/working_dir/44_tf_data_pipeline_exercise
total 16
drwxr-xr-x 3 root root 4096 Oct 24 07:51 .
drwxr-xr-x 5 root root 4096 Oct 24 07:51 ..
drwxr-xr-x 4 root root 4096 Oct 24 07:51 reviews
-rw-r--r-- 1 root root  828 Oct 24 07:51 tf_data_pipeline_exercise.md


## Load files for creating dataset

In [30]:
dataset = tf.data.Dataset.list_files('reviews/*/*', shuffle=False)
dataset

<TensorSliceDataset shapes: (), types: tf.string>

In [31]:
# Take 2 samples
for file in dataset.take(2):
  print(file.numpy())

b'reviews/negative/neg_1.txt'
b'reviews/negative/neg_2.txt'


In [32]:
len(dataset)

6

# Create data input pipeline

## 1. Read text review and generate a label from folder name. Dataset has review text and label as tuple

In [49]:
# Take all files
for file in dataset:
  print(file.numpy())

b'reviews/negative/neg_1.txt'
b'reviews/negative/neg_2.txt'
b'reviews/negative/neg_3.txt'
b'reviews/positive/pos_1.txt'
b'reviews/positive/pos_2.txt'
b'reviews/positive/pos_3.txt'


In [50]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [57]:
# Method to get labels from folder name
def get_label(file_path):
  import os
  parts = tf.strings.split(file_path, os.path.sep)
  return parts[-2]

In [63]:
# Create an empty set for storing labels of dataset
labels = set()

# Extract labels from folder names
for data in dataset:
  label = get_label(data).numpy()
  if label not in labels:
    labels.add(label)

print(labels)

{b'positive', b'negative'}


In [151]:
def text_review(filepath):
  label = get_label(filepath)
  txt_review = tf.io.read_file(filepath)
  return txt_review , label

In [152]:
# Read text review and generate a label from folder name.
# Dataset has review text and label as tuple
for data in dataset.take(1):
  txt, label = text_review(data)

print(type(txt))
# print(label)

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [153]:
review_dataset = dataset.map(text_review)

## 2. Filter blank text review

In [156]:
filtered_dataset = review_dataset.filter(c)

In [158]:
# Verify the result
for file_name, label in filtered_dataset:
  print("Text: ", file_name.numpy()[:50])
  print("Label: ", label)

Text:  b"Basically there's a family where a little boy (Jak"
Label:  tf.Tensor(b'negative', shape=(), dtype=string)
Text:  b'This show was an amazing, fresh & innovative idea '
Label:  tf.Tensor(b'negative', shape=(), dtype=string)
Text:  b'One of the other reviewers has mentioned that afte'
Label:  tf.Tensor(b'positive', shape=(), dtype=string)
Text:  b'A wonderful little production. <br /><br />The fil'
Label:  tf.Tensor(b'positive', shape=(), dtype=string)


## 3. Do all of the above transformations in single line of code.
## Also shuffle all the reviews

In [159]:
# Tensorflow data pipeline
pipeline_dataset = tf.data.Dataset.list_files('reviews/*/*', shuffle=True).map(text_review).filter(lambda txt, label: txt!="")

In [163]:
# Verify the result
for text, label in pipeline_dataset:
  print("Text: ", text.numpy()[:70])
  print("Label", label)
  print("*" * 70)

Text:  b"This show was an amazing, fresh & innovative idea in the 70's when it "
Label tf.Tensor(b'negative', shape=(), dtype=string)
**********************************************************************
Text:  b'A wonderful little production. <br /><br />The filming technique is ve'
Label tf.Tensor(b'positive', shape=(), dtype=string)
**********************************************************************
Text:  b"Basically there's a family where a little boy (Jake) thinks there's a "
Label tf.Tensor(b'negative', shape=(), dtype=string)
**********************************************************************
Text:  b'One of the other reviewers has mentioned that after watching just 1 Oz'
Label tf.Tensor(b'positive', shape=(), dtype=string)
**********************************************************************
